In [1]:
import scipy.stats as sps
import numpy as np
import matplotlib.pyplot as plt

p = 5

n = 25
n1 = 2
n2 = 6
n3 = 11
n4 = 4
n5 = 2

alpha = 0.05

## Исходная выборку

In [2]:
Psi = np.array([[1, 0, 0, 0, 0] for i in range(n1)] + 
               [[0, 1, 0, 0, 0] for i in range(n2)] +
               [[0, 0, 1, 0, 0] for i in range(n3)] +
               [[0, 0, 0, 1, 0] for i in range(n4)] +
               [[0, 0, 0, 0, 1] for i in range(n5)]) 

Y = np.atleast_2d([83, 85, 84, 85, 85, 86, 86, 87, 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90, 89, 90, 90, 91, 90, 92]).T


## а) Определить влияение возраста на содержание иммуноглобулина в крови

In [3]:
# Построим уравнение линейной регрессии

F = np.dot(Psi.T, Psi)
F_rever = np.linalg.inv(F)

betta_sample = np.dot(np.dot(F_rever, Psi.T), Y)
print("Уравнение линейной регрессии: ")
print("y = ", np.round(betta_sample[0, 0], 3), "* x_1 + ",  np.round(betta_sample[1, 0], 3), "* x_2 + ", np.round(betta_sample[2, 0], 3), "* x_3 + ", np.round(betta_sample[3, 0], 3), "* x_4 + ", np.round(betta_sample[4, 0], 3), "* x_5")


Уравнение линейной регрессии: 
y =  84.0 * x_1 +  85.5 * x_2 +  87.818 * x_3 +  90.0 * x_4 +  91.0 * x_5


In [4]:
# Определим коэффициент детерминанции

e = Y - np.dot(Psi, betta_sample)

RSS = np.dot( e.T, e )
print("RSS = ", RSS[0, 0])

y_mean = np.mean(Y)

TSS = np.sum((Y - y_mean)**2)
print("TSS = ", TSS)

R_2 = (TSS - RSS) / TSS
print("R_2 = ", R_2[0,0])


RSS =  23.136363636363637
TSS =  122.16
R_2 =  0.8106060606060606


### Таким образом, действительно, возраст влияет на содержание иммуноглобина в крови

## б) Провести попарное сравнение средних в рамках регрессионной модели, с учётом множественности проверяемых гипотез 

#### Проверим гипотезы: # 
H0: bettai = bettaj \
H1: bettai != bettaj 

delta = (~bettai - ~bettaj) * sqrt(n-p) / sqrt(RSS * (Fii_-1 + Fjj_-1)) ~ t(n-p)

In [5]:
def all_pairs(numbers):
    pairs = np.empty((0, 2))
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            pairs = np.append(pairs, [[numbers[i], numbers[j]]], axis=0)
    return pairs

pairs = all_pairs(np.arange(p))

p_values = np.empty((0,3))

for pair in pairs:
    i = int(pair[0])
    j = int(pair[1])
    delta_pair = (betta_sample[i] - betta_sample[j]) * np.sqrt(n-p) / np.sqrt(RSS * (F_rever[i, i] + F_rever[j, j]))
    p_value_pair = 2 * (1 - sps.t.cdf(abs(delta_pair), (n - p)))
    
    p_values = np.append(p_values,  [np.append(pair, p_value_pair[0 ,0])], axis=0)


p_values = p_values[p_values[:,2].argsort()]
print("p-values: ")
for pv in p_values:
    print(int(pv[0])+1, int(pv[1])+1, pv[2])

p-values: 
1 5 2.4125702768884594e-06
2 4 2.5534724907849693e-06
1 4 2.782111500732043e-06
2 5 4.085183105129175e-06
1 3 0.0001662014487755492
2 3 0.00039504010033408754
3 5 0.0010025629060046448
3 4 0.0023933387043622023
1 2 0.10310040890082384
4 5 0.29579135227474707


#### Метод Холма-Бонферрони #

In [6]:

for s in range(len(p_values)-1):
    if p_values[s, 2] < alpha / (len(p_values) - s):
        print("Отвергаем H0 для ", p_values[s, 0]+1, " и ", p_values[s, 1]+1, " bettas, те считаем, что нельзя их считать равными")
    else:
        for c in range(s, len(p_values)):
            print("Нет оснований отвергнуть H0 для ", p_values[c, 0]+1, " и ", p_values[c, 1]+1, " bettas, те считаем, что можно их считать равными")
        break

Отвергаем H0 для  1.0  и  5.0  bettas, те считаем, что нельзя их считать равными
Отвергаем H0 для  2.0  и  4.0  bettas, те считаем, что нельзя их считать равными
Отвергаем H0 для  1.0  и  4.0  bettas, те считаем, что нельзя их считать равными
Отвергаем H0 для  2.0  и  5.0  bettas, те считаем, что нельзя их считать равными
Отвергаем H0 для  1.0  и  3.0  bettas, те считаем, что нельзя их считать равными
Отвергаем H0 для  2.0  и  3.0  bettas, те считаем, что нельзя их считать равными
Отвергаем H0 для  3.0  и  5.0  bettas, те считаем, что нельзя их считать равными
Отвергаем H0 для  3.0  и  4.0  bettas, те считаем, что нельзя их считать равными
Нет оснований отвергнуть H0 для  1.0  и  2.0  bettas, те считаем, что можно их считать равными
Нет оснований отвергнуть H0 для  4.0  и  5.0  bettas, те считаем, что можно их считать равными
